## Import libaries

In [1]:
import os
import sys
assert os.path.exists('../src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements".


In [33]:
%load_ext autoreload
%autoreload 2


from src import preprocessor
from src.utils import load_documents
import nltk
import json
import ast
import pandas as pd
from time import time
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from src.bulk_runner import OUTPUT_FOLDER
from src.bulk_runner import main_runner

In [ ]:
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)

In [45]:
combinations = []

for comb_filename in ['combinations.txt','combinations_ctm.txt']:
    with open(comb_filename) as comb_file:
        for item in comb_file.readlines():
            #item = item.replace("'",'"')
            x= ast.literal_eval(item)
            combinations.append(x)

In [ ]:
    dfs_merged=pd.DataFrame()
    for i,configs in enumerate(combinations):
        print(f'[INFO] Configs #{i}/{len(combinations)}: {configs}')
        try:
            df_output_topic_word = main_runner(configs=configs)
            dfs_merged = pd.concat([dfs_merged,df_output_topic_word])
            pd.concat(dfs_merged).to_csv(f'./{OUTPUT_FOLDER}/merged.csv')
        except Exception:
            print('[WARN] Current execution gave an error!')
            time.sleep(10)
            continue


[INFO] Configs #0/1629: {'algorithm_args': {'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2', 'top_n_words': 15, 'algorithm': 'bertopic', 'n_gram_range_tuple': (1, 1), 'min_docs_per_topic': 15, 'num_topics': 4, 'cluster_model': 'hdbscan', 'hdbscan_args': {'min_cluster_size': 15, 'metric': 'euclidean', 'cluster_selection_method': 'eom', 'prediction_data': True}, 'umap_args': {'n_neighbors': 15, 'n_components': 3, 'min_dist': 0.0, 'metric': 'cosine', 'low_memory': False, 'random_state': 42}}, 'preprocessing_funcs': ['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_contractions', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_english_stop_words', 'lemmatize_noun'], 'dataset': 'crisis_12'}
[INFO] [PREPROCESSOR] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_sp

Batches:   0%|          | 0/250 [00:00<?, ?it/s]